# Lab 06-1 Softmax Classification Eager
* Softmax를 사용하여분류를 진행합니다.

### 기본 Library 선언 및 Tensorflow 버전 확인

In [2]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
tf.set_random_seed(777)  # for reproducibility
tfe = tf.contrib.eager

In [19]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [20]:
#dataset을 선언합니다.
# dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
# dataset = dataset.repeat().batch(2)

nb_classes = 3 #class의 개수입니다.

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [21]:
#Weight and bias setting
W = tfe.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-0.19601157,  0.21892233, -0.69331926],
       [-0.7571666 ,  0.62700933,  0.7786061 ],
       [-1.2000623 , -0.20798121, -2.2410874 ],
       [-0.48892695, -1.3168424 ,  1.5276555 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([ 0.14128284, -1.2531738 , -0.14682682], dtype=float32)>


In [22]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[2.9288368e-02 2.0647623e-01 7.6423538e-01]
 [7.8743957e-02 6.6927880e-01 2.5197729e-01]
 [8.8265901e-03 2.1690426e-02 9.6948302e-01]
 [1.3264089e-02 1.5685822e-01 8.2987773e-01]
 [3.5622713e-07 4.9066655e-03 9.9509299e-01]
 [1.0295569e-04 6.1157695e-04 9.9928552e-01]
 [6.2525351e-07 2.5425639e-03 9.9745685e-01]
 [5.0825658e-08 9.7215298e-04 9.9902785e-01]], shape=(8, 3), dtype=float32)


In [23]:
# Softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[0.00283937 0.03048898 0.9666716 ]], shape=(1, 3), dtype=float32)


In [24]:
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(5.915913, shape=(), dtype=float32)


In [25]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x # x^2
dy_dx = g.gradient(y, x) # Will compute to 6.0
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [26]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)

        return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: id=194966, shape=(4, 3), dtype=float32, numpy=
array([[-0.21669793, -0.46917862,  0.68587667],
       [-1.605047  , -1.0851988 ,  2.6902456 ],
       [-1.5801448 , -1.4858341 ,  3.0659792 ],
       [-1.5888716 , -1.6917062 ,  3.2805781 ]], dtype=float32)>, <tf.Tensor: id=194964, shape=(3,), dtype=float32, numpy=array([-0.23372161, -0.24208292,  0.47580457], dtype=float32)>]


In [27]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 3.223562
Loss at epoch 100: 0.730418
Loss at epoch 200: 0.612726
Loss at epoch 300: 0.549639
Loss at epoch 400: 0.498597
Loss at epoch 500: 0.451360
Loss at epoch 600: 0.405370
Loss at epoch 700: 0.359657
Loss at epoch 800: 0.313972
Loss at epoch 900: 0.269458
Loss at epoch 1000: 0.236338
Loss at epoch 1100: 0.223562
Loss at epoch 1200: 0.213007
Loss at epoch 1300: 0.203368
Loss at epoch 1400: 0.194534
Loss at epoch 1500: 0.186408
Loss at epoch 1600: 0.178909
Loss at epoch 1700: 0.171969
Loss at epoch 1800: 0.165529
Loss at epoch 1900: 0.159539
Loss at epoch 2000: 0.153952


# Prediction Check

In [12]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[0.00112885 0.08154664 0.91732454]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [16]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data

tf.Tensor(
[[6.9392156e-03 1.6807391e-01 8.2498682e-01]
 [2.1877693e-01 7.3840278e-01 4.2820331e-02]
 [2.6040227e-05 9.7818965e-01 2.1784354e-02]
 [7.4462470e-05 9.9973053e-01 1.9491679e-04]
 [8.3987463e-01 1.6012253e-01 2.9043188e-06]
 [6.4250851e-01 3.5749149e-01 1.0729331e-08]
 [9.9087501e-01 9.1250343e-03 2.2634534e-09]
 [9.9843353e-01 1.5664010e-03 2.4902956e-11]], shape=(8, 3), dtype=float32)
tf.Tensor([2 1 1 1 0 0 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


# Convert as Class

In [17]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tfe.Variable(tf.random_normal([4, nb_classes]), name='weight')
        self.b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(logits), axis=1))
        
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)
            
            return grads
    
def fit(self, X, Y, epochs=2000, verbose=500):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)

    for i in range(epochs):
        grads = self.grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, self.variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, self.loss_fn(X, Y).numpy()))
            
model = softmax_classifer(nb_classes)
model.fit(x_data, y_data)

NotImplementedError: When subclassing the `Model` class, you should implement a `call` method.